In [1]:
import matplotlib.pyplot as plt

In [2]:
import nidaqmx 
from nidaqmx.constants import Edge, AcquisitionType, TaskMode, WAIT_INFINITELY
from nidaqmx.stream_readers import AnalogMultiChannelReader
from nidaqmx.stream_writers import AnalogMultiChannelWriter
import numpy as np

In [3]:
num_samples = 12000
# total time = num_samples/sample_rate
# Inididual point writing time = 1/samplerate
sample_rate = 12000
ao_data = np.array([
    [0 if index%2==0 else 0.25 for index in range(num_samples)],
    [0 if index%2==0 else 0.25 for index in range(num_samples)],
    [0 if index%2==0 else 0.25 for index in range(num_samples)]
])

individual_point_time = 1/sample_rate


with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task, nidaqmx.Task() as pulse_task:

    ao_task.ao_channels.add_ao_voltage_chan('Dev2/ao0:2')
    ao_task.timing.cfg_samp_clk_timing(sample_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=num_samples)


    ai_task.ai_channels.add_ai_voltage_chan('Dev2/ai0:4')
    ai_task.timing.cfg_samp_clk_timing(sample_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=num_samples)
    
    pulse_channel = pulse_task.co_channels.add_co_pulse_chan_freq('Dev2/ctr0', freq=sample_rate, duty_cycle=0.25)
    pulse_channel.co_pulse_term = '/Dev2/PFI0'
    pulse_task.timing.cfg_implicit_timing(sample_mode=AcquisitionType.FINITE, samps_per_chan=num_samples)

    ai_task.triggers.start_trigger.cfg_dig_edge_start_trig(trigger_source='/Dev2/ao/StartTrigger')
    pulse_task.triggers.start_trigger.cfg_dig_edge_start_trig(trigger_source='/Dev2/ao/StartTrigger')

    ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)
    ao_writer.write_many_sample(ao_data)

    ai_task.start()
    pulse_task.start()
    ao_task.start()

    ai_reader = AnalogMultiChannelReader(ai_task.in_stream)
    ai_data = np.zeros((5, num_samples))
    ai_reader.read_many_sample(ai_data, num_samples)

    ao_task.wait_until_done()
    ai_task.wait_until_done()
    pulse_task.wait_until_done()
    

In [4]:
# num_samples = 12000
# # total time = num_samples/sample_rate
# # Inididual point writing time = 1/samplerate
# sample_rate = 12000
# ao_data = np.array([
#     [0 if index%2==0 else 0.25 for index in range(num_samples)],
#     [0 if index%2==0 else 0.25 for index in range(num_samples)],
#     [0 if index%2==0 else 0.25 for index in range(num_samples)]
# ])

# individual_point_time = 1/sample_rate

# ai_sample_factor = 8


# with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task, nidaqmx.Task() as pulse_task:

#     ao_task.ao_channels.add_ao_voltage_chan('Dev2/ao0:2')
#     ao_task.timing.cfg_samp_clk_timing(sample_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=num_samples)


#     ai_task.ai_channels.add_ai_voltage_chan('Dev2/ai0:4')
#     ai_task.timing.cfg_samp_clk_timing(sample_rate * ai_sample_factor, 
#                                        sample_mode=AcquisitionType.FINITE, 
#                                        samps_per_chan=num_samples * ai_sample_factor)
    
#     pulse_channel = pulse_task.co_channels.add_co_pulse_chan_freq('Dev2/ctr0', freq=sample_rate, duty_cycle=0.25)
#     pulse_channel.co_pulse_term = '/Dev2/PFI0'
#     pulse_task.timing.cfg_implicit_timing(sample_mode=AcquisitionType.FINITE, samps_per_chan=num_samples)

#     ai_task.triggers.start_trigger.cfg_dig_edge_start_trig(trigger_source='/Dev2/ao/StartTrigger')
#     pulse_task.triggers.start_trigger.cfg_dig_edge_start_trig(trigger_source='/Dev2/ao/StartTrigger')

#     ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)
#     ao_writer.write_many_sample(ao_data)

#     ai_task.start()
#     pulse_task.start()
#     ao_task.start()

#     ai_reader = AnalogMultiChannelReader(ai_task.in_stream)
#     ai_data = np.zeros((5, num_samples * 1))
#     ai_reader.read_many_sample(ai_data, num_samples)

#     ao_task.wait_until_done()
#     ai_task.wait_until_done()
#     pulse_task.wait_until_done()
    